# load the dataset

In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
import scipy.stats as stats
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
plt.style.use('fivethirtyeight')
from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
data = pd.read_csv(r"D:\all python works\for thesis\HepatitisCdata.csv")


data.drop([ "Unnamed: 0"], axis=1, inplace=True)
print(data.columns)

data.replace(to_replace=['0=Blood Donor', '0s=suspect Blood Donor', '1=Hepatitis',
       '2=Fibrosis', '3=Cirrhosis'],
             value=[0,1,2,3,4],inplace=True)

data.replace(to_replace=['m','f'],
             value=[0,1],inplace=True)

Index(['Category', 'Age', 'Sex', 'ALB', 'ALP', 'ALT', 'AST', 'BIL', 'CHE',
       'CHOL', 'CREA', 'GGT', 'PROT'],
      dtype='object')


# Proporsed Adaptive Data preprocessing

In [9]:

               #handling missing values of cirrosis
data_cirrhosis=data[data['Category']== 4]
from sklearn.impute import SimpleImputer
si = SimpleImputer(missing_values = np.nan, strategy ='mean')
si = si.fit(data_cirrhosis.iloc[:,1:13])
data_cirrhosis.iloc[:,1:13]= si.transform(data_cirrhosis.iloc[:,1:13])

               #for blood donor
data_blood_donor=data[data['Category']==0]
from sklearn.impute import SimpleImputer
si = SimpleImputer(missing_values = np.nan, strategy ='mean')
si = si.fit(data_blood_donor.iloc[:,1:13])
data_blood_donor.iloc[:,1:13]= si.transform(data_blood_donor.iloc[:,1:13])#set_index was not used so needed to start from column 3

               #for heaptitis
data_hepatitis=data[data['Category']==2]
from sklearn.impute import SimpleImputer
si = SimpleImputer(missing_values = np.nan, strategy ='mean')
si = si.fit(data_hepatitis.iloc[:,1:13])
data_hepatitis.iloc[:,1:13]= si.transform(data_hepatitis.iloc[:,1:13])#set_index was not used so needed to start from column 3

           #for fibrosis
data_Fibrosis=data[data['Category']==3]
from sklearn.impute import SimpleImputer
si = SimpleImputer(missing_values = np.nan, strategy ='mean')
si = si.fit(data_Fibrosis.iloc[:,1:13])
data_Fibrosis.iloc[:,1:13]= si.transform(data_Fibrosis.iloc[:,1:13])

               #for suspect blood donor
data_suspect=data[data['Category']== 1]

data=pd.concat([data_blood_donor,data_suspect,data_cirrhosis,data_Fibrosis,data_hepatitis])
data.info()

# Assuming data is your dataframe
# First, let's filter only category 0
data_category0 = data[data['Category'] == 0]

# Now, let's detect and replace outliers in each column
new_columns = [ 'Age', 'ALB', 'ALP', 'ALT', 'AST', 'BIL', 'CHE', 'CHOL', 'CREA', 'GGT', 'PROT']

outliers_exist = True
while outliers_exist:
    outliers_exist = False
    for column in new_columns:
        Q1 = data_category0[column].quantile(0.25)
        Q3 = data_category0[column].quantile(0.75)
        IQR = Q3 - Q1

        # Define outliers
        outliers = ((data_category0[column] < (Q1 - 1.5 * IQR)) | (data_category0[column] > (Q3 + 1.5 * IQR)))

        if outliers.any():
            outliers_exist = True

            # Replace outliers with the mean of the non-outliers values
            data_category0.loc[outliers, column] = data_category0.loc[~outliers, column].mean()

# Now, replace the original category 0 with the cleaned one
for index in data_category0.index:
    data.loc[index] = data_category0.loc[index]
                                       #now log transform
data['Logged ALT']=np.log(data['ALT'])
data['Logged AST']=np.log(data['AST'])
data['DLogged AST']=np.log(data['Logged AST'])
data['TLogged AST']=np.log(data['DLogged AST']) #Further log transform of AST will result the values to be negative so it will not be logtransformable
data['Logged BIL']=np.log(data['BIL'])
data['Logged GGT']=np.log(data['GGT'])
data['DLogged GGT']=np.log(data['Logged GGT']) #Further log transform of GGT will result the values to be negative so it will not be logtransformable


new_columns=['Age','Sex','ALB','CHE','CHOL','PROT','CREA','ALP','Logged ALT','TLogged AST','Logged BIL','DLogged GGT','Category']
data[new_columns]
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

# Set the style to white
sns.set_style("whitegrid")
data.drop(['ALT'], axis=1,inplace=True)
data.drop(['AST'], axis=1,inplace=True)
data.drop(['Logged AST'], axis=1,inplace=True)
data.drop(['DLogged AST'], axis=1,inplace=True)
data.drop(['BIL'], axis=1,inplace=True)
data.drop(['GGT'], axis=1,inplace=True)
data.drop(['Logged GGT'], axis=1,inplace=True)


<class 'pandas.core.frame.DataFrame'>
Index: 615 entries, 0 to 563
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Category  615 non-null    int64  
 1   Age       615 non-null    int64  
 2   Sex       615 non-null    int64  
 3   ALB       615 non-null    float64
 4   ALP       615 non-null    float64
 5   ALT       615 non-null    float64
 6   AST       615 non-null    float64
 7   BIL       615 non-null    float64
 8   CHE       615 non-null    float64
 9   CHOL      615 non-null    float64
 10  CREA      615 non-null    float64
 11  GGT       615 non-null    float64
 12  PROT      615 non-null    float64
dtypes: float64(10), int64(3)
memory usage: 67.3 KB


C:\Users\ahadk\AppData\Local\Temp\ipykernel_25068\1630033749.py:57: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '46.966037735849056' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data_category0.loc[outliers, column] = data_category0.loc[~outliers, column].mean()
C:\Users\ahadk\AppData\Local\Temp\ipykernel_25068\1630033749.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '46.966037735849056' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[index] = data_category0.loc[index]


# train model and feature selection and scaling

In [10]:
X_train=data.drop(columns=['Category','Sex','CREA'],axis=1)
y_train=data['Category']

# Standardize the features
ss = StandardScaler()
X_train = ss.fit_transform(X_train)

                  #Extract the test data and data balancing
'''

# Save the column names
column_names = X_train.columns.tolist()

# Standardize the features
ss = StandardScaler()
X_train = ss.fit_transform(X_train)

# Create an instance of the SMOTE class
smote = SMOTE()

# Generate X_resampled and y_resampled
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Selecting a specified number of samples for each category for testing
num_samples_per_category = 100  # Specify the desired number of samples for each category

test_indices = []
for category in np.unique(y_resampled):
    category_indices = np.where(y_resampled == category)[0]
    category_indices = category_indices[:num_samples_per_category]
    test_indices.extend(category_indices)

# Resetting the index of X_resampled and y_resampled
X_resampled = pd.DataFrame(X_resampled, columns=column_names)
y_resampled = pd.Series(y_resampled)

# Creating a new set of indices for test_indices
new_test_indices = []
for index in test_indices:
    new_test_indices.append(X_resampled.index[index])

# Extracting the corresponding samples for testing
X_test = X_resampled.loc[new_test_indices]
y_test = y_resampled[new_test_indices]

# Dropping the rows for testing samples from X_resampled and y_resampled
X_train = X_resampled.drop(index=new_test_indices)
y_train = y_resampled.drop(index=new_test_indices)'''


#model building
from sklearn.svm import SVC
svm = SVC()

from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
xg = XGBClassifier()

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()


from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(max_iter=10000)
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.model_selection import cross_val_score, StratifiedKFold
import numpy as np
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score, make_scorer
import xgboost as xgb
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_absolute_error
from math import sqrt
from sklearn.metrics import mean_squared_error

# Assuming you have defined and initialized the model lr, X_train, and y_train

# Create StratifiedKFold with 10 folds
#cv = StratifiedKFold(n_splits=7, shuffle=True, random_state=42)





# Assuming you have defined and initialized the models xg, X_train, and y_train

# xgBoost
xg_acc = cross_val_score(estimator=xg, X=X_train, y=y_train, cv=7, scoring='accuracy')
xg_prec_scorer = make_scorer(precision_score, average='macro', zero_division=1)
xg_prec = cross_val_score(estimator=xg, X=X_train, y=y_train, cv=7, scoring=xg_prec_scorer)
xg_rec_scorer = make_scorer(recall_score, average='macro', zero_division=1)
xg_rec = cross_val_score(estimator=xg, X=X_train, y=y_train, cv=7, scoring=xg_rec_scorer)
xg_f1_scorer = make_scorer(f1_score, average='macro', zero_division=1)
xg_f1 = cross_val_score(estimator=xg, X=X_train, y=y_train, cv=7, scoring=xg_f1_scorer)
xg_mae = cross_val_score(estimator=xg, X=X_train, y=y_train, cv=7, scoring='neg_mean_absolute_error')
xg_rmse = cross_val_score(estimator=xg, X=X_train, y=y_train, cv=7, scoring='neg_mean_squared_error')
actual_mae = abs(xg_mae.mean())
print(f"xgBoost Accuracy : {xg_acc.mean()}")
print(f"xgBoost Precision : {xg_prec.mean()}")
print(f"xgBoost Recall : {xg_rec.mean()}")
print(f"xgBoost F1-Score : {xg_f1.mean()}")
print(f"xgBoost MAE : {actual_mae.mean()}")
print(f"xgBoost RMSE : {sqrt(-xg_rmse.mean())}\n")

# SVM
svm_acc = cross_val_score(estimator=svm, X=X_train, y=y_train, cv=7, scoring='accuracy')
svm_prec_scorer = make_scorer(precision_score, average='macro', zero_division=1)
svm_prec = cross_val_score(estimator=svm, X=X_train, y=y_train, cv=7, scoring=svm_prec_scorer)
svm_rec_scorer = make_scorer(recall_score, average='macro', zero_division=1)
svm_rec = cross_val_score(estimator=svm, X=X_train, y=y_train, cv=7, scoring=svm_rec_scorer)
svm_f1_scorer = make_scorer(f1_score, average='macro', zero_division=1)
svm_f1 = cross_val_score(estimator=svm, X=X_train, y=y_train, cv=7, scoring=svm_f1_scorer)
svm_mae = cross_val_score(estimator=svm, X=X_train, y=y_train, cv=7, scoring='neg_mean_absolute_error')
svm_rmse = cross_val_score(estimator=svm, X=X_train, y=y_train, cv=7, scoring='neg_mean_squared_error')
actual_mae = abs(svm_mae.mean())
print(f"svm Accuracy : {svm_acc.mean()}")
print(f"svm Precision : {svm_prec.mean()}")
print(f"svm Recall : {svm_rec.mean()}")
print(f"svm F1-Score : {svm_f1.mean()}")
print(f"svm MAE : {actual_mae.mean()}")
print(f"svm RMSE : {sqrt(-svm_rmse.mean())}\n")

# lr
lr_acc = cross_val_score(estimator=lr, X=X_train, y=y_train, cv=7, scoring='accuracy')
lr_prec_scorer = make_scorer(precision_score, average='macro', zero_division=1)
lr_prec = cross_val_score(estimator=lr, X=X_train, y=y_train, cv=7, scoring=lr_prec_scorer)
lr_rec_scorer = make_scorer(recall_score, average='macro', zero_division=1)
lr_rec = cross_val_score(estimator=lr, X=X_train, y=y_train, cv=7, scoring=lr_rec_scorer)
lr_f1_scorer = make_scorer(f1_score, average='macro', zero_division=1)
lr_f1 = cross_val_score(estimator=lr, X=X_train, y=y_train, cv=7, scoring=lr_f1_scorer)
lr_mae = cross_val_score(estimator=lr, X=X_train, y=y_train, cv=7, scoring='neg_mean_absolute_error')
lr_rmse = cross_val_score(estimator=lr, X=X_train, y=y_train, cv=7, scoring='neg_mean_squared_error')
actual_mae = abs(lr_mae.mean())
print(f"logistic regression Accuracy : {lr_acc.mean()}")
print(f"logistic regression Precision : {lr_prec.mean()}")
print(f"logistic regression Recall : {lr_rec.mean()}")
print(f"logistic regression F1-Score : {lr_f1.mean()}")
print(f"logistic regression MAE : {actual_mae.mean()}")
print(f"logistic regression RMSE : {sqrt(-lr_rmse.mean())}\n")

rf_acc = cross_val_score(estimator=rf, X=X_train, y=y_train, cv=7, scoring='accuracy')
rf_prec_scorer = make_scorer(precision_score, average='macro', zero_division=1)
rf_prec = cross_val_score(estimator=rf, X=X_train, y=y_train, cv=7, scoring=rf_prec_scorer)
rf_rec_scorer = make_scorer(recall_score, average='macro', zero_division=1)
rf_rec = cross_val_score(estimator=rf, X=X_train, y=y_train, cv=7, scoring=rf_rec_scorer)
rf_f1_scorer = make_scorer(f1_score, average='macro', zero_division=1)
rf_f1 = cross_val_score(estimator=rf, X=X_train, y=y_train, cv=7, scoring=rf_f1_scorer)
rf_mae = cross_val_score(estimator=rf, X=X_train, y=y_train, cv=7, scoring='neg_mean_absolute_error')
rf_rmse = cross_val_score(estimator=rf, X=X_train, y=y_train, cv=7, scoring='neg_mean_squared_error')
actual_mae = abs(rf_mae.mean())
print(f"random forest Accuracy : {rf_acc.mean()}")
print(f"random forest Precision : {rf_prec.mean()}")
print(f"random forest Recall : {rf_rec.mean()}")
print(f"random forest F1-Score : {rf_f1.mean()}")
print(f"random forest MAE : {actual_mae.mean()}")
print(f"random forest RMSE : {sqrt(-rf_rmse.mean())}\n")

dt_acc = cross_val_score(estimator=dt, X=X_train, y=y_train, cv=7, scoring='accuracy')
dt_prec_scorer = make_scorer(precision_score, average='macro', zero_division=1)
dt_prec = cross_val_score(estimator=dt, X=X_train, y=y_train, cv=7, scoring=dt_prec_scorer)
dt_rec_scorer = make_scorer(recall_score, average='macro', zero_division=1)
dt_rec = cross_val_score(estimator=dt, X=X_train, y=y_train, cv=7, scoring=dt_rec_scorer)
dt_f1_scorer = make_scorer(f1_score, average='macro', zero_division=1)
dt_f1 = cross_val_score(estimator=dt, X=X_train, y=y_train, cv=7, scoring=dt_f1_scorer)
dt_mae = cross_val_score(estimator=dt, X=X_train, y=y_train, cv=7, scoring='neg_mean_absolute_error')
dt_rmse = cross_val_score(estimator=dt, X=X_train, y=y_train, cv=7, scoring='neg_mean_squared_error')
actual_mae = abs(dt_mae.mean())
print(f"decision tree Accuracy : {dt_acc.mean()}")
print(f"decision tree Precision : {dt_prec.mean()}")
print(f"decision tree Recall : {dt_rec.mean()}")
print(f"decision tree F1-Score : {dt_f1.mean()}")
print(f"decision tree MAE : {actual_mae.mean()}")
print(f"decision tree RMSE : {sqrt(-dt_rmse.mean())}\n")

xgBoost Accuracy : 0.9479773100462755
xgBoost Precision : 0.7833787640930497
xgBoost Recall : 0.6719047619047619
xgBoost F1-Score : 0.7002193448249291
xgBoost MAE : 0.07314524555903866
xgBoost RMSE : 0.358297878526625

svm Accuracy : 0.9544334975369458
svm Precision : 0.8434184862756292
svm Recall : 0.6258145363408522
svm F1-Score : 0.63115858144673
svm MAE : 0.08462083893118377
svm RMSE : 0.44197999993927367

logistic regression Accuracy : 0.962569040155247
logistic regression Precision : 0.9008740024291562
logistic regression Recall : 0.7657142857142857
logistic regression F1-Score : 0.7613955259523164
logistic regression MAE : 0.06185624720107479
logistic regression RMSE : 0.3564443086317881

random forest Accuracy : 0.9544894760412
random forest Precision : 0.7949402977974406
random forest Recall : 0.7119047619047619
random forest F1-Score : 0.7653040453718143
random forest MAE : 0.060158232572025676
random forest RMSE : 0.3123926894419564

decision tree Accuracy : 0.92353336318853

# ensample model

# ensamble logistic regression

In [11]:
rf.fit(X_train, y_train)
svm.fit(X_train, y_train)
from xgboost import XGBClassifier

xg.fit(X_train, y_train)
dt.fit(X_train, y_train)
from sklearn.linear_model import LogisticRegression
#lr = LogisticRegression(max_iter=10000)
#lr.fit(X_train, y_train)
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

estimator_list=[
    ('rf',rf),
    ('svm',svm),
    
    ('xg',xg),
    ('dt',dt)
    ]
stack_model=StackingClassifier(
    estimators=estimator_list,final_estimator=LogisticRegression(max_iter=10000)
)
stack_acc = cross_val_score(estimator=stack_model, X=X_train, y=y_train, cv=5, scoring='accuracy')
stack_prec_scorer = make_scorer(precision_score, average='macro', zero_division=1)
stack_prec = cross_val_score(estimator=stack_model, X=X_train, y=y_train, cv=5, scoring=stack_prec_scorer)
stack_rec_scorer = make_scorer(recall_score, average='macro', zero_division=1)
stack_rec = cross_val_score(estimator=stack_model, X=X_train, y=y_train, cv=5, scoring=stack_rec_scorer)
stack_f1_scorer = make_scorer(f1_score, average='macro', zero_division=1)
stack_f1 = cross_val_score(estimator=stack_model, X=X_train, y=y_train, cv=5, scoring=stack_f1_scorer)
stack_mae = cross_val_score(estimator=stack_model, X=X_train, y=y_train, cv=5, scoring='neg_mean_absolute_error')
stack_rmse = cross_val_score(estimator=stack_model, X=X_train, y=y_train, cv=5, scoring='neg_mean_squared_error')
actual_mae = abs(stack_mae.mean())
print(f"stacked model Accuracy : {stack_acc.mean()}")
print(f"stacked model Precision : {stack_prec.mean()}")
print(f"stacked model Recall : {stack_rec.mean()}")
print(f"stacked model F1-Score : {stack_f1.mean()}")
print(f"stacked MAE : {actual_mae.mean()}")
print(f"stacked RMSE : {sqrt(-stack_rmse.mean())}\n")

stacked model Accuracy : 0.9528455284552845
stacked model Precision : 0.7545291005291005
stacked model Recall : 0.6826666666666668
stacked model F1-Score : 0.7472878461585264
stacked MAE : 0.07154471544715449
stacked RMSE : 0.3651483716701107



# ensambled xgboost

In [12]:
from sklearn.svm import SVC
svm = SVC()

from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
rf.fit(X_train, y_train)
svm.fit(X_train, y_train)


dt.fit(X_train, y_train)
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression(max_iter=100000)
lr.fit(X_train, y_train)

estimator_list=[
    ('rf',rf),
    ('svm',svm),
    ('lr',lr),
    ('dt',dt)
    ]
stack_model=StackingClassifier(
    estimators=estimator_list,final_estimator= XGBClassifier()

)
stack_acc = cross_val_score(estimator=stack_model, X=X_train, y=y_train, cv=7, scoring='accuracy')
stack_prec_scorer = make_scorer(precision_score, average='macro', zero_division=1)
stack_prec = cross_val_score(estimator=stack_model, X=X_train, y=y_train, cv=7, scoring=stack_prec_scorer)
stack_rec_scorer = make_scorer(recall_score, average='macro', zero_division=1)
stack_rec = cross_val_score(estimator=stack_model, X=X_train, y=y_train, cv=7, scoring=stack_rec_scorer)
stack_f1_scorer = make_scorer(f1_score, average='macro', zero_division=1)
stack_f1 = cross_val_score(estimator=stack_model, X=X_train, y=y_train, cv=7, scoring=stack_f1_scorer)
stack_mae = cross_val_score(estimator=stack_model, X=X_train, y=y_train, cv=7, scoring='neg_mean_absolute_error')
stack_rmse = cross_val_score(estimator=stack_model, X=X_train, y=y_train, cv=7, scoring='neg_mean_squared_error')
actual_mae = abs(stack_mae.mean())
print(f"stacked model Accuracy : {stack_acc.mean()}")
print(f"stacked model Precision : {stack_prec.mean()}")
print(f"stacked model Recall : {stack_rec.mean()}")
print(f"stacked model F1-Score : {stack_f1.mean()}")
print(f"stacked MAE : {actual_mae.mean()}")
print(f"stacked RMSE : {sqrt(-stack_rmse.mean())}\n")

stacked model Accuracy : 0.9593036274070758
stacked model Precision : 0.7516388373531229
stacked model Recall : 0.7754385964912281
stacked model F1-Score : 0.8213444739855638
stacked MAE : 0.0601768920734438
stacked RMSE : 0.3228208720331013



# ensamble random forest

In [13]:
from sklearn.svm import SVC
svm = SVC()

from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
xg = XGBClassifier()
xg.fit(X_train, y_train)
from sklearn.ensemble import RandomForestClassifier


from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()

svm.fit(X_train, y_train)


dt.fit(X_train, y_train)
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression(max_iter=100000)
lr.fit(X_train, y_train)

estimator_list=[
    ('xg',xg),
    ('svm',svm),
    ('lr',lr),
    ('dt',dt)
    ]
stack_model=StackingClassifier(
    estimators=estimator_list,final_estimator= RandomForestClassifier()

)
stack_acc = cross_val_score(estimator=stack_model, X=X_train, y=y_train, cv=7, scoring='accuracy')
stack_prec_scorer = make_scorer(precision_score, average='macro', zero_division=1)
stack_prec = cross_val_score(estimator=stack_model, X=X_train, y=y_train, cv=7, scoring=stack_prec_scorer)
stack_rec_scorer = make_scorer(recall_score, average='macro', zero_division=1)
stack_rec = cross_val_score(estimator=stack_model, X=X_train, y=y_train, cv=7, scoring=stack_rec_scorer)
stack_f1_scorer = make_scorer(f1_score, average='macro', zero_division=1)
stack_f1 = cross_val_score(estimator=stack_model, X=X_train, y=y_train, cv=7, scoring=stack_f1_scorer)
stack_mae = cross_val_score(estimator=stack_model, X=X_train, y=y_train, cv=7, scoring='neg_mean_absolute_error')
stack_rmse = cross_val_score(estimator=stack_model, X=X_train, y=y_train, cv=7, scoring='neg_mean_squared_error')
actual_mae = abs(stack_mae.mean())
print(f"stacked model Accuracy : {stack_acc.mean()}")
print(f"stacked model Precision : {stack_prec.mean()}")
print(f"stacked model Recall : {stack_rec.mean()}")
print(f"stacked model F1-Score : {stack_f1.mean()}")
print(f"stacked MAE : {actual_mae.mean()}")
print(f"stacked RMSE : {sqrt(-stack_rmse.mean())}\n")

stacked model Accuracy : 0.9609456635318703
stacked model Precision : 0.8227011084153941
stacked model Recall : 0.8095238095238095
stacked model F1-Score : 0.8201904847960689
stacked MAE : 0.048831915211225554
stacked RMSE : 0.28815756334103637



# PLotting roc curve

In [14]:
# Initialize the classifiers
rf_clf = RandomForestClassifier()
xgb_clf = XGBClassifier()
dt_clf = DecisionTreeClassifier()
lr_clf = LogisticRegression(max_iter=1000)
svm_clf = SVC(probability=True)  # Set probability=True to enable probability estimates for SVC

classifiers = [rf_clf, xgb_clf, dt_clf, lr_clf, svm_clf,stack_model]
clf_names = ['Random Forest', 'XGBoost', 'Decision Tree', 'Logistic Regression', 'SVM','Stacked_model']

# Dictionary to store the ROC curve data for each model
roc_data = {name: {'fpr': [], 'tpr': [], 'roc_auc': 0.0} for name in clf_names}

# Train and evaluate each model
for clf, name in zip(classifiers, clf_names):
    clf.fit(X_train, y_train)
    y_prob = clf.predict_proba(X_test)

    # Combine the ROC curves for each class for this model
    all_fpr = np.linspace(0, 1, 100)
    mean_tpr = 0.0
    for class_idx in range(len(np.unique(y_train))):
        fpr, tpr, _ = roc_curve(y_test == class_idx, y_prob[:, class_idx])
        mean_tpr += np.interp(all_fpr, fpr, tpr)
        roc_data[name]['fpr'].append(fpr)
        roc_data[name]['tpr'].append(tpr)

    mean_tpr /= len(np.unique(y_train))
    roc_data[name]['tpr'] = mean_tpr
    roc_data[name]['roc_auc'] = auc(all_fpr, mean_tpr)

# ... (previous code up to initialization of classifiers)

# ... (rest of the code for training and evaluating each model)

plt.figure(figsize=(10, 6), facecolor='white')

# Plot the combined ROC curve for each model
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
linestyles = ['-', '-', '-', '-', '-', '-']
linewidths = [2, 2, 2, 2, 2, 2]
for name, color, linestyle, linewidth in zip(clf_names, colors, linestyles, linewidths):
    mean_fpr = np.linspace(0, 1, 100)
    mean_tpr = roc_data[name]['tpr']
    mean_auc = roc_data[name]['roc_auc']
    plt.plot(mean_fpr, mean_tpr, color=color, linestyle=linestyle, linewidth=linewidth,
             label=f'{name} ( AUC = {mean_auc:.2f})')

plt.plot([0, 1], [0, 1], color='gray', linestyle='--', linewidth=2)  # Random guessing line with adjusted width
plt.xlim([-0.02, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
#plt.title('Combined ROC Curves for Each Model')
plt.legend(loc='lower right')

# Set the background to white
fig = plt.gcf()
fig.patch.set_facecolor('white')
plt.gca().set_facecolor('white')

# Save the plot to a file
plt.savefig('roc_curves.png', dpi=1000, bbox_inches='tight', transparent=True, facecolor='white')

# Show the plot
plt.show()



NameError: name 'X_test' is not defined